## Modélisation : Projections de sièges du second tour

Le modèle proposé est très simple mais permet simplement d'illustrer le processus d'estimation du nombre de sièges en estimant les paramètres d'une matrice de report

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import dirichlet
from pathlib import Path
import geopandas as gpd
import altair as alt

## Chargement des données

In [2]:
DATA_DIR = Path().resolve().joinpath("../data/")
data = pd.read_csv(DATA_DIR.joinpath("legislatives2024/computed/data.csv"))

In [3]:
contours_circos = gpd.read_file(
    DATA_DIR.joinpath("insee/circonscriptions_legislatives_030522.shp")
)
contours_circos["is_overseas"] = contours_circos["dep"].str.len() > 2
contours_circos = contours_circos.rename(columns={"id_circo": "CodCirElec"})

In [4]:
data.head()

,Unnamed: 0,NomPsn,PrenomPsn,Departement,CodCirElec,LibCirElec,NbSap,NbSiePourvus,Inscrits,Abstentions,...,NumPanneauCand,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,GroupPol,valid_round_two
0,0,LAHY,Éric,01,0101,1ère circonscription,1,0,86843,25013,...,1,M.,EXG,Extrême gauche,419,0.69,0.48,NON,NFP+,False
1,1,MAÎTRE,Christophe,01,0101,1ère circonscription,1,0,86843,25013,...,2,M.,RN,Rassemblement National,23819,39.37,27.43,QUALIF T2,RN+,True
2,2,BRETON,Xavier,01,0101,1ère circonscription,1,0,86843,25013,...,3,M.,LR,Les Républicains,14495,23.96,16.69,QUALIF T2,LR+,True
3,3,GUERAUD,Sébastien,01,0101,1ère circonscription,1,0,86843,25013,...,4,M.,UG,Union de la gauche,14188,23.45,16.34,QUALIF T2,NFP+,False
4,4,VINCENT,Cyril,01,0101,1ère circonscription,1,0,86843,25013,...,5,M.,DSV,Droite souverainiste,197,0.33,0.23,NON,RN+,False


In [5]:
first_round_seats = (
    data[data["Elu"] == "OUI"]["GroupPol"]
    .value_counts()
    .reset_index()
    .rename(columns={"count": "Sièges", "GroupPol": "Nuance politique"})
    .set_index("Nuance politique")
)
first_round_seats.loc["DIV", "Sièges"] = 0
first_round_seats

,Sièges
Nuance politique,
RN+,39.0
NFP+,32.0
LR+,3.0
ENS+,2.0
DIV,0.0


In [6]:
circos_out = data[data["Elu"] == "OUI"]["CodCirElec"].unique()

In [7]:
print(f"{len(circos_out)} circonscriptions déjà pourvues")

76 circonscriptions déjà pourvues


In [8]:
# Regroupement des candidats "Divers"
data["NbVoix"] = data.groupby(["CodCirElec", "GroupPol", "valid_round_two"])[
    "NbVoix"
].transform(sum)
data = data.drop_duplicates(["CodCirElec", "GroupPol", "valid_round_two"])

/tmp/ipykernel_346074/473034050.py:4: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ].transform(sum)


#### Visualisation des réservoirs de voix

Combien y a-t-il d'électeurs ayant voté pour des candidats ayant perdu (qui vont donc se reporter ou s'abstenir).

In [9]:
group_to_color = {
    "RN+": "#0b5394",
    "DIV": "#8e7cc3",
    "LR+": "#0086ff",
    "NFP+": "#e06666",
    "ENS+": "#f1c232",
}

In [10]:
alt.Chart(
    data[~data["valid_round_two"]].groupby("GroupPol")["NbVoix"].sum().reset_index()
).mark_arc().encode(
    theta=alt.Theta("NbVoix:Q"),
    color=alt.Color("GroupPol:N").scale(
        domain=list(group_to_color.keys()), range=list(group_to_color.values())
    ),
).properties(title="Réservoirs de voix par \"ensemble politique\"")

alt.Chart(...)

Afin de pouvoir effectuer des projections en sièges, il est nécessaire de modéliser le comportement des électeurs des différents partis politiques, en particulier de ceux dont le parti n'est pas représenté au 2nd tour (non qualifié ou désistement du candidat)

### Estimation des probabilités de report

Les hypothèses choisies sont les suivantes : 
- Un électeur ayant voté au 1er tour pour un parti d'un bord politique votera pour le même bord politique au 2nd tour si son bord politique est représenté (ex.: un électeur ayant voté LFI Dissident au 1er tour votera UG au 2nd s'il s'agit du seul candidat de gauche)
- Si le bord politique de l'électeur n'est pas représenté au 2nd tour, celui-ci se reportera vers d'autres partis ou s'abstiendra. Plus le bord politique du parti représenté est proche de celui de l'électeur au 1er tour, plus la probabilité que l'électeur se reporte vers ce parti sera élevée.
- Un électeur qui s'est abstenu au 1er tour s'abstiendra au 2nd tour
- Les probabilités de report sont symétriques (parti A vers B = parti B vers A)

Ces hypothèses ont notamment pour conséquence une abstention strictement supérieure à celle du 1er tour

En conséquence, on considère dans un premier temps une matrice de probabilités de report qui dépend de 9 paramètres : 
- $\alpha_1$ : La probabilité de passer de `NFP+` à `ENS+` ou de `ENS+` à `LR+` 
- $\alpha_2$ : La probabilité de passer de `RN+` à `LR+`
- $\alpha_3$ : La probabilité de passer de `NFP+` à `RN+` (ou de `RN+` à `NFP+`) ou de `ENS+` à `RN+`
- $\alpha_4$ : La probabilité de passer de `NFP+` à `LR+`
- $\alpha_5, \alpha_6, \alpha_7, \alpha_8$ les taux de report de `DIV` vers les autres partis
- $\alpha_9$ un taux d'abstention minimum

<img src="../img/matrice_reports.png" alt="Matrice de reports" width="400"/>

Cette matrice ne représente pas directement une matrice de probabilités, car la matrice de probabilités dépend de la configuration (voir l'exemple ci-dessous)

Pour rappel les bords politiques sont déterminés comme suit :
- `RN+` = `UXD` | `RN` | `DSV`
- `NFP+` = `UG` | `COM` | `ECO` | `PS`
- `LR+` = `LR` | `DVD`
- `ENS+` = `ENS` | `HOR`

#### Exemple : La 1ère circonscription de l'Ain


In [11]:
data[data["CodCirElec"] == "0101"]

,Unnamed: 0,NomPsn,PrenomPsn,Departement,CodCirElec,LibCirElec,NbSap,NbSiePourvus,Inscrits,Abstentions,...,NumPanneauCand,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,GroupPol,valid_round_two
0,0,LAHY,Éric,01,0101,1ère circonscription,1,0,86843,25013,...,1,M.,EXG,Extrême gauche,14607,0.69,0.48,NON,NFP+,False
1,1,MAÎTRE,Christophe,01,0101,1ère circonscription,1,0,86843,25013,...,2,M.,RN,Rassemblement National,23819,39.37,27.43,QUALIF T2,RN+,True
2,2,BRETON,Xavier,01,0101,1ère circonscription,1,0,86843,25013,...,3,M.,LR,Les Républicains,14495,23.96,16.69,QUALIF T2,LR+,True
4,4,VINCENT,Cyril,01,0101,1ère circonscription,1,0,86843,25013,...,5,M.,DSV,Droite souverainiste,511,0.33,0.23,NON,RN+,False
6,6,GUILLERMIN,Vincent,01,0101,1ère circonscription,1,0,86843,25013,...,7,M.,ENS,Ensemble ! (Majorité présidentielle),7063,11.68,8.13,NON,ENS+,False


La matrice de probabilités de report est la suivante : 
|   | DIV  | ENS+  | LR+  | NFP+  | RN+ | ABS |
|---|---|---|---|---|---| ---|
| DIV  | 0  | 0|  $\frac{\alpha_5}{\alpha_5+\alpha_7+\alpha_8}$  | 0  | $\frac{\alpha_7}{\alpha_5+\alpha_7+\alpha_8}$ | $\frac{\alpha_8}{\alpha_5+\alpha_7+\alpha_8}$ |
| ENS+ |  0 |  0 | $\frac{\alpha_1}{\alpha_1+\alpha_3+\alpha_8}$  | 0  | $\frac{\alpha_3}{\alpha_5+\alpha_7+\alpha_8}$ | $\frac{\alpha_8}{\alpha_5+\alpha_7+\alpha_8}$ |
| LR+ | 0  |  0 |  1 |  0 | 0| 0|
| NFP+ | 0  | 0  | $\frac{\alpha_2}{\alpha_2+\alpha_3+\alpha_8}$  | 0  |$\frac{\alpha_3}{\alpha_2+\alpha_3+\alpha_8}$|$\frac{\alpha_8}{\alpha_2+\alpha_3+\alpha_8}$ | 
| RN+ | 0  | 0  | 0  | 0  |1 | 0| 
| ABS | 0  | 0  | 0 | 0 | 0 | 1 |


Ainsi les résultats pourraient être les suivants

|   | L  | M  | R  | FR  | Abs | Total (1er tour) |
|---|---|---|---|---|---| ---|
| L  | 0  | 0|  4382  | 730  | 9995 | 14607 |
| M |  0 |  0 | 4238  | 353  | 2472 | 7063 |
| R | 0  |  0 |  14495 |  0 | 0| 14495|
| FR | 0  | 0  | 0  | 23819  |0 |23819 | 
| Abs | 0  | 0  | 0  | 0  |25013 | 25013| 
| Total (2e tour) | 0  | 0  | 23115  (48.1%)| 24902 (51.9%) | 37480 (43%)| 86843|


### Constitution des matrices de calcul

Les matrices utilisées sont les suivantes : 
- $M_{RET}$ la matrice $(n_{circos}, n_{ensembles})=(501,5)$ avec les scores du 1er tour des candidats présents au 2nd tour
- $M_{DISQ}$ la matrice $(n_{circos}, n_{ensembles})=(501,5)$ avec les scores du 1er tour des candidats non présents au 2nd tour
- $T$ est la matrice de reports $(n_{ensembles}+1, n_{ensembles}+1) = (6,6)$ qui inclut l'abstention.


In [12]:
p1, p2, p3, p4 = 0.6, 0.3, 0.1, 0.3
p5, p6, p7, p8 = 0.3, 0.3, 0.3, 0.3
p_abs = 0.3
transfer_matrix = np.array(
    [
        [1, p5, p6, p7, p8, p_abs],
        [p5, 1, p1, p1, p3, p_abs],
        [p6, p1, 1, p2, p4, p_abs],
        [p7, p1, p2, 1, p3, p_abs],
        [p8, p3, p4, p3, 1, p_abs],
        [0, 0, 0, 0, 0, 1],

    ]
)

In [13]:
abstentions = (
    data[~data["CodCirElec"].isin(circos_out)]
    .drop_duplicates(["CodCirElec", "Abstentions"])["Abstentions"]
    .values
)

In [14]:
second_round_pivoted = (
    data[(data["valid_round_two"]) & (~data["CodCirElec"].isin(circos_out))]
    .pivot(columns=["GroupPol"], index="CodCirElec", values=["NbVoix"])
    .fillna(0)
)
print(second_round_pivoted.head())
second_round_matrix = second_round_pivoted.values

           NbVoix                                    
GroupPol      DIV     ENS+      LR+     NFP+      RN+
CodCirElec                                           
0101          0.0      0.0  14495.0      0.0  23819.0
0102          0.0  17414.0      0.0      0.0  28189.0
0103          0.0  17420.0      0.0      0.0  17252.0
0104          0.0  14367.0      0.0      0.0  30221.0
0105          0.0      0.0      0.0  12542.0  20161.0


In [15]:
second_round_matrix = np.hstack([second_round_matrix, abstentions[:, None]])

In [16]:
index_to_political_group = {i: col[1] for i, col in enumerate(second_round_pivoted.columns)}

#### Constitution de la matrice des réservoirs de vote

In [17]:
remaining_votes = (
    data[(~data["valid_round_two"]) & (~data["CodCirElec"].isin(circos_out))]
    .pivot(columns=["GroupPol"], index="CodCirElec", values=["NbVoix"])
    .fillna(0)
)

In [18]:
remaining_votes_matrix = remaining_votes.values
remaining_votes_matrix = np.hstack([remaining_votes_matrix, np.zeros(len(second_round_matrix))[:, None]])

In [19]:
second_round_mask = (second_round_matrix > 0).astype(int)
second_round_mask.shape

(501, 6)

#### Exemple du calcul détaillé pour la première circonscription

Matrice de report normalisée

In [20]:
parameters_for_circo = (
    (1 - np.tile(second_round_mask[0], (6, 1)).T)
    * transfer_matrix
    * np.tile(second_round_mask[0], (6, 1))
)
normalized_parameters = np.nan_to_num(parameters_for_circo / parameters_for_circo.sum(-1)[:, None], 0)

/tmp/ipykernel_346074/619680807.py:6: RuntimeWarning: invalid value encountered in divide
  normalized_parameters = np.nan_to_num(parameters_for_circo / parameters_for_circo.sum(-1)[:, None], 0)


Echantillonnage suivant une loi multinomiale

In [21]:
samples = np.zeros((6,6))
for i in range(6):
    samples[i] = (np.random.multinomial(n=remaining_votes_matrix[0,i], pvals=normalized_parameters[i]))
samples

array([[   0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0., 4229.,    0.,  745., 2089.],
       [   0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0., 6245.,    0., 2126., 6236.],
       [   0.,    0.,    0.,    0.,    0.,  511.],
       [   0.,    0.,    0.,    0.,    0.,    0.]])

Résultats (la dernière colonne représente l'abstention)

In [22]:
samples.sum(0)

array([    0.,     0., 10474.,     0.,  2871.,  8836.])

Auxquels on ajoute les voix des candidats élus au 1er tour

In [23]:
samples.sum(0) + second_round_matrix[0]

array([    0.,     0., 24969.,     0., 26690., 33849.])

In [24]:
from scipy.stats import truncnorm


def sample_transfer_matrix(s=0.3, size=1):
    params = [0.6, 0.3, 0.1, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
    sampled_params = [
        truncnorm.rvs(a=-param / s, b=(1 - param) / s, loc=param, scale=s, size=size)
        for param in params
    ]
    return np.array(
        [
            [
                np.ones(size),
                sampled_params[4],
                sampled_params[5],
                sampled_params[6],
                sampled_params[7],
                sampled_params[8],
            ],
            [
                sampled_params[4],
                np.ones(size),
                sampled_params[0],
                sampled_params[0],
                sampled_params[2],
                sampled_params[8],
            ],
            [
                sampled_params[5],
                sampled_params[0],
                np.ones(size),
                sampled_params[1],
                sampled_params[3],
                sampled_params[8],
            ],
            [
                sampled_params[6],
                sampled_params[0],
                sampled_params[1],
                np.ones(size),
                sampled_params[2],
                sampled_params[8],
            ],
            [
                sampled_params[7],
                sampled_params[2],
                sampled_params[3],
                sampled_params[2],
                np.ones(size),
                sampled_params[8],
            ],
            [
                np.zeros(size),
                np.zeros(size),
                np.zeros(size),
                np.zeros(size),
                np.zeros(size),
                np.ones(size),
            ],
        ]
    )

In [25]:
def sample_circo(i, n_simus=1, precomputed_transition_matrix=None):
    if precomputed_transition_matrix is None:
        precomputed_transition_matrix = sample_transfer_matrix(size=n_simus)
    full_binary_mask = np.tile(second_round_mask[i], (6, 1))
    parameters_for_circo = (
        (1 - full_binary_mask.T)[:, :, None]
        * precomputed_transition_matrix
        * full_binary_mask[:, :, None]
    )
    normalized_parameters = np.nan_to_num(
        parameters_for_circo / parameters_for_circo.sum(1)[:, None, :], 0
    )

    samples = np.zeros((6, 6, n_simus))

    for party in range(6):
        for i_simu in range(n_simus):
            samples[party, :, i_simu] = np.random.multinomial(
                n=remaining_votes_matrix[i, party],
                pvals=normalized_parameters[party, :, i_simu],
            )
    return samples.sum(0) + second_round_matrix[i, :, None]

In [26]:
def simulate_single_run():
    results = np.zeros((len(second_round_matrix), 6))
    for i in range(len(second_round_matrix)):
        results[i] = sample_circo(i)[:, 0]

    return results


In [27]:
simulation_results = simulate_single_run()

/tmp/ipykernel_346074/2222412437.py:11: RuntimeWarning: invalid value encountered in divide
  parameters_for_circo / parameters_for_circo.sum(1)[:, None, :], 0


Il suffit ensuite de compter le nombre de circonscriptions dans lequel chaque parti gagne (sauf l'abstention !)

In [28]:
labels, total_seats =  np.unique(simulation_results[:,:-1].argmax(1), return_counts=True)
predictions = {
    index_to_political_group[label]: int(seats)
    for label, seats in zip(labels, total_seats)
}
predictions

{'DIV': 10, 'ENS+': 106, 'LR+': 32, 'NFP+': 155, 'RN+': 198}

### Simulation Monte Carlo

Il est en réalité très complexe de connaître la vraie valeur des paramètres, c'est pourquoi il est important de les considérer comme des variables aléatoires et d'échantillonner plusieurs scénarios.

Note : Il serait possible d'optimiser sous forme matricielle l'échantillonnage pour accélérer le processus...

In [29]:
sample_circo(0, n_simus=2)

/tmp/ipykernel_346074/2222412437.py:11: RuntimeWarning: invalid value encountered in divide
  parameters_for_circo / parameters_for_circo.sum(1)[:, None, :], 0


array([[    0.,     0.],
       [    0.,     0.],
       [23549., 18274.],
       [    0.,     0.],
       [30404., 31465.],
       [31555., 35769.]])

In [30]:
def simulate_multiple_runs(n_simus=10):
    results = np.zeros((len(second_round_matrix), 6, n_simus))
    for i in range(len(second_round_matrix)):
        results[i] = sample_circo(i, n_simus)
    return results

In [31]:
n_simus = 1000
results = simulate_multiple_runs(n_simus=n_simus)

/tmp/ipykernel_346074/2222412437.py:11: RuntimeWarning: invalid value encountered in divide
  parameters_for_circo / parameters_for_circo.sum(1)[:, None, :], 0


In [32]:
def results_to_frame(results):
    """
    Conversion des résultats de numpy array en pandas dataframe
    """
    results_df = pd.DataFrame()
    for i in range(results.shape[-1]):
        tmp_df = pd.DataFrame(
            results[:, :, i],
            columns=[index_to_political_group[i] for i in range(5)] + ["ABS"],
            index=second_round_pivoted.index,
        )
        tmp_df["id_simu"] = i
        results_df = pd.concat([results_df, tmp_df])
    return results_df

In [33]:
df_results = results_to_frame(results).reset_index()

In [ ]:
melted_results = df_results.melt(
    ["id_simu", "CodCirElec"], var_name="Nuance politique", value_name="NbVoix"
)

### Analyse des projections

Calcul des gagnants par circonscription

In [ ]:
winners = melted_results[melted_results["Nuance politique"] != "ABS"].sort_values(
    "NbVoix", ascending=False
).drop_duplicates(
    ["CodCirElec", "id_simu"]
)
winners.head()

,id_simu,CodCirElec,Nuance politique,NbVoix
736706,470,5601,ENS+,61485.0
695624,388,5601,ENS+,61131.0
985703,967,5601,ENS+,61125.0
827387,651,5601,ENS+,60990.0
889010,774,5601,ENS+,60641.0


In [ ]:
seats = winners.groupby("id_simu")["Nuance politique"].value_counts().reset_index()
seats.head()

,id_simu,Nuance politique,count
0,0,RN+,182
1,0,NFP+,158
2,0,ENS+,116
3,0,LR+,38
4,0,DIV,7


In [ ]:
seats.groupby("Nuance politique")["count"].agg(
    [
        ("mean", "mean"),
        ("quantile_0.1", lambda x: np.quantile(x, 0.1)),
        ("quantile_0.9", lambda x: np.quantile(x, 0.9)),
    ]
)

,mean,quantile_0.1,quantile_0.9
Nuance politique,,,
DIV,8.994,8.0,10.0
ENS+,117.147,111.0,123.0
LR+,35.364,32.0,39.0
NFP+,159.263,154.0,165.0
RN+,180.232,172.0,188.0


On constate que compte-tenu de l'incertitude élevée sur les paramètres, les intervalles de confiance sont particulièrement importants. On voit notamment que l'incertitude la plus importante porte sur le nombre de circonscriptions d'Ensemble

In [ ]:
wins_by_circo = (
    winners.groupby("CodCirElec")["Nuance politique"].value_counts() / n_simus
)
uncertainty_by_circo = (
    wins_by_circo.reset_index()
    .sort_values("count", ascending=False)
    .drop_duplicates("CodCirElec")
    .rename(columns={"count": "% victoires"})
)
uncertainty_by_circo.tail()

,CodCirElec,Nuance politique,% victoires
107,1702,NFP+,0.533
575,6911,ENS+,0.532
94,1406,ENS+,0.532
124,2101,ENS+,0.526
174,2704,NFP+,0.504


In [ ]:
data[data["CodCirElec"] == "8501"]

,Unnamed: 0,NomPsn,PrenomPsn,Departement,CodCirElec,LibCirElec,NbSap,NbSiePourvus,Inscrits,Abstentions,...,NumPanneauCand,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,GroupPol,valid_round_two
3113,3113,ETONNO,Lucie,85,8501,1ère circonscription,1,0,120711,36734,...,1,Mme,UG,Union de la gauche,19871,23.33,15.68,QUALIF T2,NFP+,False
3115,3115,BARIAL,Jean-Marc,85,8501,1ère circonscription,1,0,120711,36734,...,3,M.,DSV,Droite souverainiste,1433,1.77,1.19,NON,RN+,False
3116,3116,LATOMBE,Philippe,85,8501,1ère circonscription,1,0,120711,36734,...,4,M.,ENS,Ensemble ! (Majorité présidentielle),23136,28.51,19.17,QUALIF T2,ENS+,True
3117,3117,CAILLAUD,Laurent,85,8501,1ère circonscription,1,0,120711,36734,...,5,M.,DVC,Divers centre,10606,13.07,8.79,NON,ENS+,False
3118,3118,PAULIN,Simon-Pierre,85,8501,1ère circonscription,1,0,120711,36734,...,6,M.,UXD,Union de l'extrême droite,26105,32.17,21.63,QUALIF T2,RN+,True


In [ ]:
melted_results[melted_results["CodCirElec"] == "8501"]

,id_simu,CodCirElec,Nuance politique,NbVoix
406,0,8501,DIV,0.0
907,1,8501,DIV,0.0
1408,2,8501,DIV,0.0
1909,3,8501,DIV,0.0
2410,4,8501,DIV,0.0
...,...,...,...,...
3003901,995,8501,ABS,53940.0
3004402,996,8501,ABS,53113.0
3004903,997,8501,ABS,58305.0
3005404,998,8501,ABS,53104.0


In [ ]:
import altair as alt
main_chart = alt.Chart(melted_results[melted_results["CodCirElec"] == "8501"])

main_chart.mark_boxplot().encode(
    x=alt.X("Nuance politique:N").title("Ensemble politique"),
    y=alt.Y("NbVoix:Q").title("Nombre de voix estimé"),
).properties(title="Résultats des simulations sur la circonscription 8501")

Nous pouvons ensuite visualiser les circonscriptions les plus incertaines (après avoir effectué une renormalisation)

In [ ]:
contours_circos["uncertainty"] = contours_circos["CodCirElec"].map(
    (1- uncertainty_by_circo.set_index("CodCirElec")["% victoires"]) * 2
)

In [ ]:
import altair as alt

selector = alt.selection_single(fields=["CodCirElec"], on="click")
alt.Chart(contours_circos[~contours_circos["is_overseas"]]).mark_geoshape().encode(
    tooltip=["CodCirElec:N", "uncertainty:Q"], color="uncertainty:Q"
).properties(
    title="Visualisation de l'incertitude dans la simulation par circonscription",
    height=500
)